# Hyperparameter Tuning using HyperDrive

### Create workspace and experiment instances

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
experiment_name = 'capstone-spam-classification-experiment'

experiment=Experiment(ws, experiment_name)

### Connect to compute cluster target

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
cluster_name = "capstone-compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Get Data

In [4]:
from azureml.core import Dataset
training_dataset = Dataset.get_by_name(ws, name='capstone-spam-dataset')

### View the dataset

In [5]:
training_dataset.to_pandas_dataframe()

,Category,Message,Column3
0,ham,"Go until jurong point, crazy.. Available only ...",None
1,ham,Ok lar... Joking wif u oni...,None
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,None
3,ham,U dun say so early hor... U c already then say...,None
4,ham,"Nah I don't think he goes to usf, he lives aro...",None
...,...,...,...
5569,spam,This is the 2nd time we have tried 2 contact u...,None
5570,ham,Will ü b going to esplanade fr home?,None
5571,ham,"Pity, * was in mood for that. So...any other s...",None
5572,ham,The guy did some bitching but I acted like i'd...,None


### Hyperdrive Configuration
We are using a Logistic Regression classifier, the main hyperparameters there are C and max_iter. 

In [6]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
from azureml.train.hyperdrive.policy import BanditPolicy
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.sampling import RandomParameterSampling
param_sampling = RandomParameterSampling( 
    {
        "C": choice(0.01, 0.001, 0.1, 1),
        "max_iter": choice(50,100),
    }
)

# Setup environment for your training run
from azureml.core import Environment
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')


# Create a ScriptRunConfig Object to specify the configuration details of your training job
from azureml.core import ScriptRunConfig
script_run_config = ScriptRunConfig(source_directory=".",
                      script='train.py',
                      compute_target=compute_target,
                      environment=sklearn_env)

# Create Hyperdrive config
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
hyperdrive_run_config = HyperDriveConfig(run_config=script_run_config,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name='Accuracy',
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=10,
                             max_concurrent_runs=4)

### Submit your hyperdrive run to the experiment 


In [7]:
hyperdrive_run = experiment.submit(hyperdrive_run_config)

### Run Details

In [13]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show() 
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_353966e4-eddf-47b6-b19d-22d440935562
Web View: https://ml.azure.com/runs/HD_353966e4-eddf-47b6-b19d-22d440935562?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-208897/workspaces/quick-starts-ws-208897&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Execution Summary
RunId: HD_353966e4-eddf-47b6-b19d-22d440935562
Web View: https://ml.azure.com/runs/HD_353966e4-eddf-47b6-b19d-22d440935562?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-208897/workspaces/quick-starts-ws-208897&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_353966e4-eddf-47b6-b19d-22d440935562',
 'target': 'capstone-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-09-26T07:32:52.37356Z',
 'endTimeUtc': '2022-09-26T07:39:30.182358Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '61d9f2d5-297e-4a25-9c50-7e73d12aeec2',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1017-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.44.0',
  'space_size': '8',
  'score': '0.9691535150645624',
  'best_child_run_id': 'HD_353966e4-eddf-47b6-b19d-22d440935562_4',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_353966e4-eddf-47b6-b19d-22d440935562_4'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {

KeyError: 'log_files'

### Get the best model and display its properties

In [9]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Best Accuracy:', best_run_metrics['Accuracy'])

Best Run Id:  HD_353966e4-eddf-47b6-b19d-22d440935562_4

 Best Accuracy: 0.9691535150645624


### Register the best model

In [10]:
model = best_run.register_model(
    model_name="hyperdrive-best-model", model_path="."
)

### Shutdown the computes

In [12]:
try:
    instance = ComputeTarget(workspace=ws, name=cluster_name)

    instance.delete()
    instance.wait_for_completion(show_output=True)
    print('Deleted compute resource')

except ComputeTargetException as e:
    print('Already deleted!')

Already deleted!
